In [1]:
from data import preparation
import data.hierarchy as hie
from sklearn.model_selection import train_test_split

In [2]:
data_name = "wipo_c"

In [3]:
with open('data/%s/hierarchy.txt' % data_name, 'w') as f1:
    with open('data/%s/%s.ht' % (data_name,data_name)) as f:
        for l in f:
            split = l.strip().split(':')
            p = split[0]
            c = split[1].split(',')
            for cc in c:
                f1.write('%s %s\n'% (p,cc))

In [4]:
hierarchy, parent_of, all_name, name_to_index, level = hie.reindex_hierarchy(
                '%s/hierarchy.txt' % data_name)
hie.save_hierarchy("%s/hierarchy.pickle" % data_name, hierarchy,
                   parent_of, all_name, name_to_index, level)

In [5]:
for mode in ['train', 'test']:
    for i in range(1,6):
        datas, labels = preparation.import_data('%s/folds/%s_fold%d.dat.%s' % (data_name, data_name, i, mode))
        new_labels = preparation.map_index_of_label('%s/hierarchy.pickle' % data_name, labels)
        if mode == 'train':
            train_data, validate_data, train_target, validate_target = train_test_split(
                datas, new_labels, test_size=0.1, random_state=12345)
            preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, mode), train_data, train_target)
            preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, "validate"), validate_data, validate_target)
        else:
            preparation.save_data_in_pickle('%s/fold/data_%d.pickle.%s' % (data_name, i, mode), datas, new_labels)

In [6]:
for i in range(len(level)-1):
    print(level[i+1] - level[i])

17
56
852
4741


# Real Flow 

In [7]:
from data import Dataset
from embedding import Doc2Vec, NoTag_Doc2Vec, OnlyLeafDoc2Vec
from assemble_classifier import AssembleNoLabel, AssemblePredicted,AssemblePredictedHidden ,AssemblePredictedHiddenAdd
import shutil
import os

%load_ext autoreload
%autoreload 2

In [8]:
shl_M = []
shl_m = []
new_M = []
new_m = []

In [9]:
shl_f1 = []
new_f1 = []

In [10]:
%%time
for fold in range(1, 6):
    print()
    print()

    dataset_train = Dataset(data_name, fold, "train")
    dataset_validate = Dataset(data_name, fold, "validate")
    dataset_test = Dataset(data_name, fold, "test")

    print("train doc2vec")
    doc2vec = OnlyLeafDoc2Vec(dataset_train.number_of_classes(), size=150, epoch=180, batch_size=10000)
    doc2vec.fit(dataset_train.datas, dataset_train.labels, dataset_validate.datas, dataset_validate.labels, early_stopping=False, one_time=True)

    dataset_train.change_to_Doc2Vec(doc2vec)
    dataset_validate.change_to_Doc2Vec(doc2vec)
    dataset_test.change_to_Doc2Vec(doc2vec)
    
    print("train main classifier")
    if os.path.isdir('data/%s/output' % data_name):
        shutil.rmtree('data/%s/output' % data_name)
#     model = AssemblePredicted(data_name, dataset_train, dataset_validate, dataset_test, iteration=2000, stopping_time=200, batch_size=65536, hidden_size=[300,1500,2000,2500], target_hidden_size=[60,100,150], use_dropout=True, start_level=0)
    model = AssemblePredictedHidden(data_name, dataset_train, dataset_validate, dataset_test, iteration=20000, batch_size=2000, hidden_size=[300,1500,1500,1500], use_dropout=True, start_level=0, end_level=100)
    model.train(verbose=False)
    model.tuning_threshold()
    print("evaluate")
    f1_macro, f1_micro, f1_each = model.evaluate("test", correction=True)
    shl_M.append(f1_macro)
    shl_m.append(f1_micro)
    shl_f1.append(f1_each)
    print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
    for level, (macro, micro) in enumerate(f1_each):
        print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))
        
#     print("train main classifier")
#     if os.path.isdir('data/%s/output' % data_name):
#         shutil.rmtree('data/%s/output' % data_name)
#     model = AssemblePredicted(data_name, dataset_train, dataset_validate, dataset_test, iteration=2000, stopping_time=200, batch_size=65536, hidden_size=[300,1500,1500,1500], target_hidden_size=[60,60,60], use_dropout=True, start_level=0)
# #     model = AssemblePredictedHidden(data_name, dataset_train, dataset_validate, dataset_test, iteration=20000, batch_size=2000, hidden_size=[300,1500,2000,2500], use_dropout=True, start_level=0, end_level=100)
#     model.train(verbose=False)
#     model.tuning_threshold()
#     print("evaluate")
#     f1_macro, f1_micro, f1_each = model.evaluate("test", correction=True)
#     new_M.append(f1_macro)
#     new_m.append(f1_micro)
#     new_f1.append(f1_each)
#     print("F1 macro: %.4f F1 micro: %.4f" % (f1_macro, f1_micro))
#     for level, (macro, micro) in enumerate(f1_each):
#         print("Level: %d F1 macro: %.4f F1 micro: %.4f" % (level, macro, micro))
     



train doc2vec
Doc2Vec by Gensim
0.125
Epoch: 180 Similar: 0.12380
train main classifier
evaluate
F1 macro: 0.0328 F1 micro: 0.3636
Level: 0 F1 macro: 0.5958 F1 micro: 0.6506
Level: 1 F1 macro: 0.4233 F1 micro: 0.4715
Level: 2 F1 macro: 0.0810 F1 micro: 0.2058
Level: 3 F1 macro: 0.0175 F1 micro: 0.1202


train doc2vec
Doc2Vec by Gensim
0.125
Epoch: 180 Similar: 0.12715
train main classifier
evaluate
F1 macro: 0.0310 F1 micro: 0.3773
Level: 0 F1 macro: 0.5972 F1 micro: 0.6550
Level: 1 F1 macro: 0.4286 F1 micro: 0.4882
Level: 2 F1 macro: 0.0783 F1 micro: 0.2331
Level: 3 F1 macro: 0.0157 F1 micro: 0.1051


train doc2vec
Doc2Vec by Gensim
0.125
Epoch: 180 Similar: 0.12305
train main classifier
evaluate
F1 macro: 0.0305 F1 micro: 0.3576
Level: 0 F1 macro: 0.5700 F1 micro: 0.6456
Level: 1 F1 macro: 0.4131 F1 micro: 0.4604
Level: 2 F1 macro: 0.0790 F1 micro: 0.2025
Level: 3 F1 macro: 0.0153 F1 micro: 0.1079


train doc2vec
Doc2Vec by Gensim
0.125
Epoch: 180 Similar: 0.13159
train main classi

In [11]:
import numpy as np
each_M = np.array([0.] * 4)
each_m = np.array([0.] * 4)
for i in new_f1:
    t = np.array(i).transpose()
    each_M += t[0]
    each_m += t[1]
print(each_M / 5)
print(each_m / 5)

[0. 0. 0. 0.]
[0. 0. 0. 0.]


In [12]:
import numpy as np
each_M = np.array([0.] * 4)
each_m = np.array([0.] * 4)
for i in shl_f1:
    t = np.array(i).transpose()
    each_M += t[0]
    each_m += t[1]
print(each_M / 5)
print(each_m / 5)

[0.58202691 0.41477769 0.07942368 0.01594262]
[0.64905866 0.47355759 0.2169923  0.11074485]


In [13]:
print(sum(shl_m) / 5)
print(sum(new_m) / 5)

0.3682063591279386
0.0


In [14]:
print(sum(shl_M) / 5)
print(sum(new_M) / 5)

0.03112865760922432
0.0
